# NFL RECEIVING YARDAGE PREDICTOR

Big data is becoming a big part of our world and sports are no execption. Teams are increasingly using advanced analytics to find an edge over their competition.

On such example of this is using modeling to find out what metrics are most predictive of future success. For example, what are the elements of wide receiver play in the NFL that are most important? 

**Problem to be solved:** Do past receiving yards do a good job of predicting future receiving yards or are there other metrics that can do a better job?

**Key Metrics**

*   Receiving yards -- how many yards a player gains 
*   Targets -- how often the player is thrown the ball
*   Completed passs/completion/reception -- when a player successfully catches a target
*   Air Yards -- how far in the air the ball travels before the player attempts to catch the ball
*   aDOT -- Average depth of target, or air yards per target
*   Yardage after catch (YAC) -- the yardage the receiver gains after the catch
*   Implied team total -- how many points a player's team is projected to score in the given game; this is derived from betting markets


**Methodology**


1.  Build a backwards-looking expected yardage model, based on inputs such as air yardage and field position.
2.   Assess how this model and/or other factors predict future receiving yards.

**Data Sources**

*   *nflverse (https://nflverse.nflverse.com/)*
*   *nfl_data_py (https://github.com/cooperdff/nfl_data_py)*












## 1. Data Prep

In [1]:
#install nfl_data_py package
!pip install nfl_data_py


In [2]:
import nfl_data_py as nfl


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
import nfl_data_py as nfl

%matplotlib inline

C:\Users\src15\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
C:\Users\src15\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\src15\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [4]:
#Calculate the market-implied team total, derived the betting lines

def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [5]:
#load in ten years worth of data for NFL pass attempts

all_throws = []

for YEAR in tqdm(range(2012,2022)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)
    
    #we only want regular season data
    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    
    
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    #derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    
    
    #we only want throws, aka plays with air yardage (no running plays, sacks, etc.)
    throws = data[data['air_yards'].isna()==False]
    
    df = throws[['receiver_player_name','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit']]
    all_throws.append(df)
    all_throws_df=pd.concat(all_throws)






100%|██████████| 10/10 [08:59<00:00, 53.98s/it]


## 2. Play-by-Play Yardage Predictor

In [6]:
#Drop throws without a listed targeted receiver
all_throws = all_throws_df.dropna(subset=['receiver_player_name'])

#Create column for throws to the middle of the field(1 for throws to middle, 0 for throws elsewhere)                               
all_throws['middle'] = (all_throws['pass_location']=='middle')*1                      

<ipython-input-6-5acd7fa4e22f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_throws['middle'] = (all_throws['pass_location']=='middle')*1


We now have all the relevant features for our model. They are the following:


*   **Down** (whether a play comes on first, second, third or fourth down)
*   **Yardline_100** (where on the field a play begins)
*   **Yards to go** (how many yards a team needs for a new first down or touchdown)
*   **Implied team total**
*   **Air yards**
*   **Middle**
*   **QB_hit** (Whether or not the passer was hit while throwing)

The target variable is **yards_gained**.






In [7]:
model_df = all_throws[['yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit']]

model_df.corr()[['yards_gained']].sort_values('yards_gained',ascending=False)

,yards_gained
yards_gained,1.000000
air_yards,0.250699
yardline_100,0.113662
middle,0.066762
implied_posteam_total,0.050664
ydstogo,0.044456
down,-0.023398
qb_hit,-0.048197


In [8]:

X = model_df.drop(columns={'yards_gained'})
y = model_df['yards_gained']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


In [10]:
#hyper paramaters tuned via gridsearch
from sklearn.ensemble import GradientBoostingRegressor
play_by_play_model= GradientBoostingRegressor(max_depth=5, min_samples_split=1500, n_estimators=50).fit(X_train, y_train)

In [11]:
play_by_play_model.score(X_train, y_train)


0.11654928097137618

In [12]:
play_by_play_model.score(X_test, y_test)

0.09722665951433684

In [13]:
import pickle
pickle.dump(play_by_play_model, open('play_by_play.pkl', 'wb'))


In [14]:
feature_df = pd.DataFrame(play_by_play_model.feature_importances_)
feature_list = []

for feature in X.columns:
    feature_list.append(feature)

feature_df['Feature'] = feature_list
feature_df.rename(columns = {0:'Importance'},inplace=True)

feature_df = feature_df[['Feature','Importance']].sort_values('Importance',ascending = False)

feature_df.sort_values('Importance',ascending=False)

,Feature,Importance
4,air_yards,0.632208
1,yardline_100,0.212436
3,implied_posteam_total,0.045707
5,middle,0.043798
6,qb_hit,0.040041
0,down,0.013637
2,ydstogo,0.012173


The model tells us that air yards are by far the most important element of predicting how many yards a pass will gain, followed by field position.


param_grid = {'n_estimators':range(10,100,20),
             'max_depth':range(1,21,4), 'min_samples_split':range(500,2001,500)}

gsearch1 = sklearn.model_selection.GridSearchCV(GradientBoostingRegressor(), param_grid, cv=3,refit = True, verbose = 3)

gsearch1.fit(X_train, y_train)

print("\n The best parameters across ALL searched params:\n",gsearch1.best_params_)

print("\n The best parameters across ALL searched params:\n",gsearch1.best_params_)

parameters = {'learning_rate': [0.0001, 0.001, 0.01],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [25, 50, 100,500,1000],
                  'max_depth'    : [1,2,5,10],
              'min_samples_split':[100,500,1000,1500]
                 }

grid_GBR = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid = parameters, cv = 2, n_jobs=-1,verbose=3)

grid_GBR.fit(X_train, y_train)


print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

In [15]:
all_throws['xYards']=play_by_play_model.predict(X)


<ipython-input-15-3dd7d5a2b3df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_throws['xYards']=play_by_play_model.predict(X)


In [16]:
all_throws['xYards'].corr(all_throws['yards_gained'])

0.33463690029530535

In [17]:
all_throws['xYards'].corr(all_throws['yards_gained'])> 0.3334010562798367

True

In [18]:
#look at receivers for this year
all_throws = []

for YEAR in range(2022,2023):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','yards_after_catch','xyac_mean_yardage','cp','cpoe']]
    all_throws.append(df)
    current_szn_df=pd.concat(all_throws)


In [19]:
current_szn_df = current_szn_df.dropna(subset=['receiver_player_name'])

#Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 

In [20]:
current_szn_df = current_szn_df[['receiver_player_name','week','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit']].set_index(['receiver_player_name','posteam'])

In [21]:
current_x = current_szn_df.drop(['yards_gained','week'],axis=1)

In [22]:
current_szn_df['xYards']=play_by_play_model.predict(current_x)
current_szn_df = current_szn_df.reset_index()

receivers = current_szn_df.groupby(['receiver_player_name','posteam']).agg({'down':'count','air_yards':'sum','yards_gained':'sum','xYards':'sum','week':'nunique'})
                                                                            
                                                                            
                                                                            #'complete_pass':'sum','cp':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum'})

receivers.rename(columns={'down':'targets','week':'games'},inplace=True)

In [23]:
receivers['Diff'] = (receivers['yards_gained'] - receivers['xYards']).round(1)

print('First 10 weeks of the 2022 season:')
print('Correlation between actual yards and predicted yards gained for receivers: {}'.format((receivers['yards_gained'].corr(receivers['xYards']).round(2))))

First 10 weeks of the 2022 season:
Correlation between actual yards and predicted yards gained for receivers: 0.98


### 2a. Leaders, Overachievers and Underachievers

In [24]:
receivers.sort_values('xYards',ascending=False).head(10)[['games','targets','yards_gained','xYards','Diff']].round(1)

,,games,targets,yards_gained,xYards,Diff
receiver_player_name,posteam,,,,,
T.Hill,MIA,14,152,1529.0,1392.8,136.2
D.Adams,LV,14,151,1275.0,1282.6,-7.6
J.Jefferson,MIN,14,157,1622.0,1211.5,410.5
S.Diggs,BUF,14,142,1299.0,1199.3,99.7
A.Brown,PHI,14,118,1201.0,1028.1,172.9
C.Lamb,DAL,14,124,1087.0,1024.7,62.3
J.Waddle,MIA,14,101,1117.0,963.8,153.2
M.Evans,TB,13,108,898.0,955.6,-57.6
T.Kelce,KC,14,127,1144.0,935.4,208.6


In [25]:
receivers.sort_values('Diff',ascending=False).head(10)[['games','targets','yards_gained','xYards','Diff']].round(1)

,,games,targets,yards_gained,xYards,Diff
receiver_player_name,posteam,,,,,
J.Jefferson,MIN,14,157,1622.0,1211.5,410.5
T.Kelce,KC,14,127,1144.0,935.4,208.6
D.Henry,TEN,12,38,379.0,204.0,175.0
G.Wilson,NYJ,14,110,966.0,792.4,173.6
A.Brown,PHI,14,118,1201.0,1028.1,172.9
R.Shaheed,NO,8,19,334.0,172.3,161.7
T.McLaurin,WAS,14,104,1015.0,859.7,155.3
D.Goedert,PHI,9,53,544.0,390.4,153.6
J.Waddle,MIA,14,101,1117.0,963.8,153.2


In [26]:
receivers.sort_values('Diff',ascending=True).head(10)[['targets','yards_gained','xYards','Diff']]

,,targets,yards_gained,xYards,Diff
receiver_player_name,posteam,,,,
A.Green,ARI,40,128.0,295.401746,-167.4
K.Pitts,ATL,59,356.0,513.229581,-157.2
S.Miller,TB,37,177.0,319.178818,-142.2
M.Jones,JAX,64,430.0,567.414694,-137.4
M.Gallup,DAL,57,339.0,469.813284,-130.8
M.Brown,ARI,88,584.0,714.526157,-130.5
D.Johnson,PIT,123,735.0,862.253532,-127.3
D.London,ATL,92,603.0,716.185827,-113.2
T.Thornton,NE,33,166.0,278.634116,-112.6


## 3. Predicting future performance

Now that we have a model for expected yardage for each play, we now should see how this predicts future performance, while also looking at other factors that predict future performance.

### 3. Year n to Year n+1

In [27]:
#load in ten years worth of data for NFL pass attempts

all_throws = []
n_szn = []
plus_1_szn = []

for YEAR in tqdm(range(2012,2022)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','cp','xyac_mean_yardage','qb_hit','yards_after_catch']]
  
  #apply model to the season's stats

    current_szn_df = df.dropna(subset=['receiver_player_name'])

#Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
    current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 
    current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1 


    current_szn_df = current_szn_df[['receiver_player_name','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit','complete_pass','yards_after_catch','cp','xyac_mean_yardage','goal_to_go']].set_index(['receiver_player_name','posteam'])
    current_x = current_szn_df.drop(['yards_gained','goal_to_go','complete_pass','yards_after_catch','cp','xyac_mean_yardage'],axis=1)
    current_szn_df['xYards']=play_by_play_model.predict(current_x)
    current_szn_df = current_szn_df.reset_index()

#create seperate DF for YAC, looking only at completed passes; then merge this back with main DF
    yac_df = current_szn_df[current_szn_df['complete_pass']==1]
    yac_df = yac_df.groupby(['receiver_player_name','posteam']).sum()
    yac_df['yac/rec'] = yac_df['yards_after_catch']/yac_df['complete_pass']
    yac_df['xyac/rec'] = yac_df['xyac_mean_yardage']/yac_df['complete_pass']
    yac_df =yac_df[['xyac/rec','yac/rec','yards_after_catch','xyac_mean_yardage']]

    receivers = current_szn_df.groupby(['receiver_player_name','posteam']).agg({'down':'count','air_yards':'mean','yards_gained':'sum','xYards':'sum','complete_pass':'sum','cp':'mean','qb_hit':'mean','yardline_100':'mean','goal_to_go':'mean'})
    receivers['catch_rate'] = receivers['complete_pass']/receivers['down']                                                                         




    receivers.rename(columns={'down':'targets'},inplace=True)



    receivers['Diff'] = receivers['yards_gained'] - receivers['xYards']
    receivers = receivers.merge(yac_df,right_index=True,left_index=True)


    receivers['Szn'] = YEAR
  
  
  
  
  
  
  
    n_szn.append(receivers)



    n_szn_df=pd.concat(n_szn)
    n_minus_one_df = pd.concat(n_szn)
    n_minus_one_df['Szn'] = n_minus_one_df['Szn'] + 1
    n_minus_one_df.set_index('Szn',append=True,inplace=True)
    n_szn_df.set_index('Szn',append=True,inplace=True)



  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-27-820c24fb5291>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1
<ipython-input-27-820c24fb5291>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1
 10%|█         | 1/10 [00:55<08:15, 55.11s/it]<ipython-input-27-820c24fb5291>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

100%|██████████| 10/10 [08:59<00:00, 53.93s/it]


In [28]:
n_minus_one_df = n_minus_one_df[n_minus_one_df['targets']>=50]

n_szn_df = n_szn_df[n_szn_df['targets']>=50]

In [29]:
df = n_minus_one_df.merge(n_szn_df,left_index=True,right_index=True)

In [30]:


df['cpoe_x'] = df['catch_rate_x'] - df['cp_x']

df['cpoe_y'] = df['catch_rate_y'] - df['cp_y']

#df

In [31]:
season_n_list = []
for col in df.columns:
    if "_x" in col:
        season_n_list.append(col)
        
season_n_list.append('yards_gained_y')

In [32]:
df[season_n_list].corr().sort_values('yards_gained_y',ascending=False)[['yards_gained_y']].round(2)

,yards_gained_y
yards_gained_y,1.00
yards_gained_x,0.55
xYards_x,0.53
targets_x,0.47
complete_pass_x,0.43
air_yards_x,0.35
xyac_mean_yardage_x,0.24
cpoe_x,0.23
Diff_x,0.18
yards_after_catch_x,0.16


*Among previous season stats (denoted with the suffix '_x'), actual yardage gained and expected yardage gained have a similar correlation with next season yardage gained.*

**What explains the difference between xYards and actual yardage gained?**

In [33]:
df['xYards/target_x'] = df['xYards_x']/df['targets_x']
df['xYards/target_y'] = df['xYards_y']/df['targets_y']

df['Yards/target_x'] = df['yards_gained_x']/df['targets_x']
df['Yards/target_y'] = df['yards_gained_y']/df['targets_y']

df['yac_oe_x'] = df['yac/rec_x'] - df['xyac/rec_x']
df['yac_oe_y'] = df['yac/rec_y'] - df['xyac/rec_y']

In [34]:
df.corr().sort_values('Diff_y',ascending=False)[['Diff_y']]

,Diff_y
Diff_y,1.000000
Yards/target_y,0.712706
cpoe_y,0.618604
yards_after_catch_y,0.551820
yac_oe_y,0.530838
catch_rate_y,0.519284
yards_gained_y,0.444222
yac/rec_y,0.413409
xyac_mean_yardage_y,0.372428
complete_pass_y,0.338030


**Year-to-year stability of various metrics**

In [35]:
metric_list = ['yards_gained','xYards','xYards/target','Yards/target','yac_oe','qb_hit','Diff','complete_pass','catch_rate','targets','air_yards','cp','cpoe','xyac/rec','yac/rec']

In [36]:
corr_dict = {}


for metric in metric_list:
  x = metric + '_x'
  y = metric + '_y'
  r = round(df[x].corr(df[y]),2)
  corr_dict.update({metric:r})


In [37]:
pd.DataFrame([corr_dict]).transpose().sort_values(0,ascending=False).rename(columns={0:'r'})

,r
air_yards,0.91
cp,0.88
xyac/rec,0.85
xYards/target,0.84
yac/rec,0.69
catch_rate,0.62
xYards,0.60
yards_gained,0.55
targets,0.51
complete_pass,0.49


**There is more stability in expected yardage from one year to the next (r=0.60) than actual yardage (0.55).** The delta between the two metrics ("Diff") is not very stable from one year to the next (0.21).*

This makese sence, since air yards are among the most stable metric from one year to the next (r=0.91), and air yards are the most important element of expected yards and expected yards per target (air yards also are a big part of other metrics with high stability, such as completion probability (CP) and xYards after catch per reception).

Also, metrics like completion percentage above expectation* (CPOE) and yards-after-catch above expecation* (YAC_OE) that make up much of the in-season difference between xYards and actual yards are not as stable from one year to the next (between season n and season n+1, r=0.38 for yac_oe, .29 for CPOE).


*CPOE and YAC_OE derive from the nflverse's built-in expected completion and expected YAC models. Further reading: https://www.opensourcefootball.com/posts/2020-09-28-nflfastr-ep-wp-and-cp-models/*

**Other conclusions:**

-A receiver's ability to earn targets seems more consistent than their ability to turn targets into catches (the year-to-year correlation for targets is 0.51, while the year-to-year correlation for completion percentage over expectation is just 0.29).

-Air yards are considerably more stable than yardage after the catch (r=0.91 for air yards year-over-year, compared to a mark of 0.38 for yards after catch above expectation).

**-Therefore, measuring "opportunity" itself (namely, the number of targets and the expected value of these targets) seems more important to making predictions than actually measuring what a receiver does with this opportunity.**

### In-season predictions

In this section, I will break down the last 10 season into six-week segments and see how well data in these segments predicts the following week.

In [38]:
import warnings
warnings.filterwarnings('ignore')

import tqdm.notebook as tq


In [39]:
#correct a few formatting errors in the data

def last_name(full):
  if full == 'DanielThomas':
    return 'Thomas'
  elif full == 'JulioJones':
    return 'Jones'
  elif '.' not in full:
    print(full)
    return full
  else:
    return full.split(".",1)[1]

In [40]:
#fix team abbreviations for teams that have moved cities

def team_fixer(team):
    if team == 'SD':
        return 'LAC'
    elif team == 'OAK':
        return 'LV'
    elif team == 'STL':
        return 'LA'
    else:
        return team

In [41]:
def snap_name_changer(player):
    if 'Jr.' in player:
        return player.split(' Jr.')[0]
        #return 'D.J. Davis'
    elif 'II' in player:
        return player.split(' II')[0]
    else:
        return player

In [42]:
#load in ten years worth of data for NFL pass attempts

szn_receivers=[]
target_list=[]
  
df_list = []
weekly_list = []
target_week = [7,8,9,10,11,12,13,14,15,16,17,18]
week_list1= []
target_weeks=[]


all_throws = []
n_szn = []
plus_1_szn = []

transposed_df = []
real_yardage_list = []

for YEAR in tqdm(range(2012,2022)):
    print(YEAR)
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','week','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','season','home_implied_total','away_implied_total','posteam_type','cp','xyac_mean_yardage','qb_hit','yards_after_catch','receiver_player_id']]

    current_szn_df = df.dropna(subset=['receiver_player_name'])

    #Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
    current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 
    current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1 

    current_szn_df = current_szn_df[['receiver_player_name','week','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit','complete_pass','yards_after_catch','cp','xyac_mean_yardage','goal_to_go','receiver_player_id']].set_index(['receiver_player_name','posteam'])
    current_szn_df['year'] = YEAR
  
    #apply model to the season's stats
    current_x = current_szn_df.drop(['yards_gained','year','week','goal_to_go','complete_pass','yards_after_catch','cp','xyac_mean_yardage','receiver_player_id'],axis=1)
    current_szn_df['xYards']=play_by_play_model.predict(current_x)
    current_szn_df = current_szn_df.reset_index()
  
    #create seperate DF for YAC, looking only at completed passes; then merge this back with main DF
    yac_df = current_szn_df[current_szn_df['complete_pass']==1]
    weekly_yac = yac_df.groupby(['receiver_player_name','posteam','week']).agg({'complete_pass':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum'})
    
    
    #create new DF for team pass attempts to calculate metrics such as target share (targets per team attempts)
    weekly_pass_offense=current_szn_df.groupby(['posteam','week']).agg({'week':'count','air_yards':'sum'})
    weekly_pass_offense.rename(columns={'week':'team_attempts','air_yards':'team_air_yards'},inplace=True)
    team_attempts=weekly_pass_offense.reset_index()[['posteam','week','team_attempts','team_air_yards']]
    weekly_rec_group = current_szn_df.groupby(['receiver_player_name','posteam','week']).agg({'week':'max','down':'count','complete_pass':'sum','yards_gained':'sum','air_yards':'sum','xYards':'sum','cp':'sum','implied_posteam_total':'max','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum','receiver_player_id':'max'})
    weekly_rec_group.rename(columns={'week':'week_#','down':'targets'},inplace=True)
    weekly_rec_group = weekly_rec_group.merge(weekly_yac[['yards_after_catch','xyac_mean_yardage']],right_index=True,left_index=True)
    
    
    feature_receivers = weekly_rec_group.reset_index()

    feature_receivers.reset_index().set_index(['receiver_player_id','posteam','week'])
    
    week_df1 = feature_receivers.copy()
    week_df = week_df1.copy()
    week_df = week_df.reset_index().set_index(['receiver_player_name','posteam'])
    feature_df = feature_receivers.copy()
    
    
    #load in snap count data; this is important because players who go entire games without a target will not show up in the data for that game
    snap_df = nfl.import_snap_counts([YEAR])
    snap_df = snap_df[snap_df['position']!='QB']
    
    snap_df['team'] = snap_df['team'].apply(lambda x: team_fixer(x))
    snap_df = snap_df[snap_df['game_type']=='REG']
    snap_df = snap_df[['pfr_player_id','player','offense_snaps','offense_pct','week','team','game_id','position']]
    snap_df['player_copy'] = snap_df['player']
    snap_df.rename(columns={'player_copy':'full_name'},inplace=True)
    snap_df['full_name'] = snap_df['full_name'].apply(lambda x: snap_name_changer(x))
    
    
    snap_df['week#'] = snap_df['week']
    snap_df = snap_df[snap_df['offense_snaps']>0]

    #create a unique identifier for each player so the snap data dataframe can be merged with the main receiver dataframe
    snap_df['last_name'] = snap_df['full_name'].apply(lambda x: x.split(" ",1)[1])
    snap_df['identifier'] = (snap_df['full_name']).str[0]+"."+snap_df['last_name']+"_"+snap_df['team']+"_"+snap_df['week#'].astype(str)
    
    feature_df_snaps = feature_df.reset_index()
    feature_df_snaps.rename(columns={'posteam':'team'},inplace=True)
    feature_df_snaps['szn'] = YEAR
    
    feature_df_snaps = feature_df_snaps.set_index(['receiver_player_id','team','week'])        
    for player_id in tq.tqdm(feature_df_snaps.index):
        player = player_id[0:2]
        team = player[1]
        week_df = pd.DataFrame()
        
        week_df['week_#']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
        new_df = feature_df_snaps.loc[player].merge(week_df,how='right')
        new_df['target_week'] = new_df['week_#']+1
        new_df['szn'] = new_df['szn'].fillna(YEAR)
        new_df['receiver_player_name'] = new_df['receiver_player_name'].fillna(new_df['receiver_player_name'].mode()[0])
        new_df['posteam'] = team
        
        new_df['identifier'] = (new_df['receiver_player_name']).str[0]+"."+new_df['receiver_player_name']+"_"+new_df['posteam']+"_"+new_df['week_#'].astype(str)
        new_df['last'] = new_df['receiver_player_name'].apply(lambda x: last_name(x))
        new_df['identifier'] = (new_df['receiver_player_name']).str[0]+"."+new_df['last']+"_"+new_df['posteam']+"_"+new_df['week_#'].astype(str)
               
        new_df = new_df.merge(snap_df[['identifier','full_name','game_id','offense_snaps','offense_pct','position']], on='identifier')
            
        #team_attempts = team_attempts.reset_index()
        
        team_attempts.rename(columns={'week':'week_#'},inplace=True)

        new_df = new_df.merge(team_attempts,on=['posteam','week_#'])
        
        new_df.fillna(0,inplace=True)


        #new_df will be the dataframe from trailing week data; 
        #the trailing period will be at a minimum three weeks worth of games and a max of six weeks
        
        
        #a copy of this dataframe will be for the target weeks
        target_df = new_df.copy()
        #print('shape: {}'.format(new_df.shape[0]))
        #if new_df.shape[0]==3:
         #weights = np.array([0.2, 0.3, 0.5])
         #new_df['weighted_xYards'] = new_df['xYards'].rolling(3).apply(lambda x: np.sum(weights*x))
         #new_df['weights'] = weights
         #new_df['unweighted_mean'] = new_df['xYards'].rolling(3,min_periods=3).mean()
         #new_df['unweighted_mean_test'] = new_df['xYards'].rolling(3,min_periods=3).mean()

          #display(new_df[['receiver_player_name','xYards','weighted_xYards','unweighted_mean','weights']])

        #if new_df.shape[0]==4:
         # weights = np.array([0.05,0.15, 0.3, 0.5])
         # new_df['weighted_xYards'] = new_df['xYards'].rolling(4).apply(lambda x: np.sum(weights*x))
          #new_df['weights'] = weights
         # new_df['unweighted_mean'] = new_df['xYards'].rolling(4,min_periods=3).mean()
         # new_df['unweighted_mean1'] = new_df['xYards'].rolling(4,min_periods=3).mean()
          #display(new_df[['receiver_player_name','xYards','weighted_xYards','unweighted_mean','unweighted_mean1']])

        #if new_df.shape[0]==5:
         # weights = np.array([0.05,0.05,0.15, 0.25, 0.5])
          #new_df['weighted_xYards'] = new_df['xYards'].rolling(5).apply(lambda x: np.sum(weights*x))
          #new_df['weights'] = weights
          #new_df['unweighted_mean'] = new_df['xYards'].rolling(5,min_periods=3).mean()
          #new_df['unweighted_mean1'] = new_df['xYards'].rolling(5,min_periods=3).mean()
          #display(new_df[['receiver_player_name','xYards','weighted_xYards','unweighted_mean','unweighted_mean1']])



          #display(new_df[['receiver_player_name','xYards','weighted_xYards','unweighted_mean']])

        



        #calculate six week-rolling averages for key metrics



        new_df = new_df.sort_values('week_#')
        
        
        
        target_df1 = pd.DataFrame()
        target_df1['target_week']=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]


        new_df = new_df.merge(target_df1,how='right').sort_values('target_week')
        #rec_name = list(new_df[new_df['receiver_player_name'].isna()==False]['receiver_player_name'])[0]
        
        

        
        checker = new_df.copy()
        if new_df.shape[0]>5:
            for name in new_df['full_name'].unique():
                player_disp = new_df[new_df['full_name']==name]
                for x in range(7,19):
                    new_p_df = player_disp[player_disp['target_week']<=x].tail(6)[['xYards']]
                    new_p_df.rename(columns={'xYards':player[0]},inplace=True)
                    real = player_disp[player_disp['target_week']<=x].tail(6)[['yards_gained']]
                    real.rename(columns={'yards_gained':player[0]},inplace=True)
                    tr = new_p_df.transpose()
                    real_df= real.transpose()
                    if len(list(tr.columns))==6:
                        tr.columns = [1,2,3,4,5,6]
                        real_df.columns = ['1a','2a','3a','4a','5a','6a']
                        if x in list(player_disp['week_#']):
                            yards = player_disp.set_index('week_#').loc[x]['yards_gained']
                            name = player_disp.set_index('week_#').loc[x]['full_name']
                            pos = player_disp.set_index('week_#').loc[x]['position']

                            #t_week = player_disp.set_index('week_#').loc[x]['target_week']
                                                                        
                            tr['yards_gained']=yards
                            tr['full_name']=name
                            tr['id']=player[0]
                            tr['week']=x
                            tr['season']=YEAR
                            tr['position']=pos
                            

                            real_df['full_name']=name
                            real_df['id']=player[0]
                            real_df['week']=x
                            real_df['season']=YEAR
                            
                            

                        transposed_df.append(tr)
                        real_yardage_list.append(real_df)

        #new_df = new_df.fillna(method='ffill')
        
        #target_list.append(target_df)
        
        
        
        

training_data = pd.concat(transposed_df)
real_yardage = pd.concat(real_yardage_list)

#target_weeks = pd.concat(target_list)   

#target_weeks = target_weeks[target_weeks['week_#']>3]
#target_weeks = target_weeks.dropna()
#target_weeks = target_weeks.drop(columns={'target_week'})
#target_weeks.rename(columns={'week_#':'target_week'},inplace=True)


#target_weeks = target_weeks.drop_duplicates()
#trailing_weeks = trailing_weeks.drop_duplicates()


#trailing_weeks = trailing_weeks[['receiver_player_name','posteam','szn','position','target_week','snaps_sum','games_played','xYards','xYards_sum','xYards/game','xYards_median','Yards/game','air_yards_rolling','team_attempts_rolling','team_air_yards_rolling','complete_pass_rolling','cp_rolling','yac_rolling','xyac_rolling','targets_sum','weighted_xYards','unweighted_xmean','unweighted_yards_mean','weighted_yards']]
#target_weeks = target_weeks[['receiver_player_name','posteam','target_week','szn','yards_gained','offense_snaps','targets','complete_pass','xYards','cp','yards_after_catch','xyac_mean_yardage']]


#target_weeks = target_weeks[target_weeks['target_week']>3]
#trailing_weeks.set_index(['receiver_player_name','posteam','target_week','szn'],inplace=True)
#target_weeks.set_index(['receiver_player_name','posteam','target_week','szn'],inplace=True)





  0%|          | 0/10 [00:00<?, ?it/s]

2012


  0%|          | 0/3589 [00:00<?, ?it/s]

 10%|█         | 1/10 [13:05<1:57:53, 785.93s/it]

2013


  0%|          | 0/3603 [00:00<?, ?it/s]

 10%|█         | 1/10 [34:11<5:07:44, 2051.66s/it]


KeyboardInterrupt: 

In [ ]:
the_don = training_data[training_data['full_name']=='Zach Pascal']

the_don.drop_duplicates().shape

In [ ]:
xyard_df = training_data.drop_duplicates()
actual_yard_df = real_yardage.drop_duplicates()

xyard_df.shape

In [ ]:
training_data_ = xyard_df.merge(actual_yard_df,on=['full_name','week','season'])

Before applying these findings to a model, we should first look at the difference between different positions.

In [ ]:
def position_handler(pos):
  if pos in ['WR','RB','TE','FB']:
    return pos
  elif pos == 'RB/W':
    return 'RB'
  elif pos == 'WR/R':
    return 'WR'
  elif pos == 'HB':
    return 'RB'
  else:
    return 'other'

In [ ]:
training_data_['position'] = training_data_['position'].apply(lambda x: position_handler(x))

In [ ]:
training_data_.head()

In [ ]:
for pos in training_data_['position'].unique():
  positional_model = training_data_[training_data_['position']==pos]
  yards_per_game = round(positional_model['yards_gained'].mean(),1)
  print('{}: {}'.format(pos, yards_per_game))



Wide receivers gain much more yardage on average than other positions, so the model should account for this.

Also, since non-WR/TE/RB/FB targets are rare, we should drop them to ensure the model is more generizable.

In [ ]:
training_data_ = training_data_.drop_duplicates()

In [ ]:
training_data_

In [ ]:
#one hot-encode position

one_hot = pd.get_dummies(training_data_['position'])
merged_model = training_data_.drop('position',axis = 1)

merged_model = merged_model.join(one_hot)

#drop non WR/TE/RB/FB
merged_model = merged_model[merged_model['other']==0]

In [ ]:
merged_model

In [ ]:
merged_model = merged_model.drop_duplicates()

merged_model = merged_model.dropna()

In [ ]:
merged_model['xYards/game']=(merged_model[1]+merged_model[2]+merged_model[3]+merged_model[4]+merged_model[5]+merged_model[6])/6

In [ ]:
merged_model['yards/game']=(merged_model['1a']+merged_model['2a']+merged_model['3a']+merged_model['4a']+merged_model['5a']+merged_model['6a'])/6

In [ ]:
#X = merged_model[[1,2,3,4,5,6,'FB','RB','WR','1a','2a','3a','4a','5a','6a']]
X = merged_model[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']]

y = merged_model['yards_gained']
linear_features = X.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
linear_model = LinearRegression().fit(X_train, y_train)



#print('xYards per game w/ positions:',in_season_model.score(X_test, y_test).round(3))
print('xYards mean:',linear_model.score(X_test, y_test).round(3))

In [ ]:
#tune max bins maybe

X = merged_model[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']]
#X = merged_model[[1,2,3,4,5,6]]

#X = merged_model[[1,2,3,4,5,6,'xYards/game']]

#X = merged_model[['xYards/game','Yards/game']].dropna()
#X = merged_model[['xYards/game','RB','WR','FB']].dropna()
#X = merged_model[['xYards_median']].dropna()

y = merged_model['yards_gained']
features = X.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
#in_season_model = GradientBoostingRegressor(max_depth=1,min_samples_split=2, n_estimators=100).fit(X_train, y_train)
in_season_model = GradientBoostingRegressor().fit(X_train, y_train)
#in_season_model = GradientBoostingRegressor(learning_rate=0.01,max_depth=4,n_estimators=500,subsample=0.5).fit(X_train, y_train)



#print('xYards per game w/ positions:',in_season_model.score(X_test, y_test).round(3))
print('xYards mean:',in_season_model.score(X_test, y_test).round(3))

In [ ]:
print('r^2:',in_season_model.score(X_test, y_test).round(3))


feature_df = pd.DataFrame(in_season_model.feature_importances_)
feature_list = []

for feature in X.columns:
    feature_list.append(feature)

feature_df['Feature'] = feature_list
feature_df.rename(columns = {0:'Importance'},inplace=True)

feature_df = feature_df[['Feature','Importance']].sort_values('Importance',ascending = False)

feature_df.sort_values('Importance',ascending=False)

In [ ]:
pickle.dump(in_season_model, open('beta_model.pkl', 'wb'))


In [61]:
parameters = {'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }

In [62]:
grid_GBR = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid = parameters, cv = 2, n_jobs=-1,verbose=3)

grid_GBR.fit(X_train, y_train)


print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

Fitting 2 folds for each of 384 candidates, totalling 768 fits


KeyboardInterrupt: 

In [ ]:
print('xYards mean:',grid_GBR.score(X_test, y_test).round(3))

In [ ]:
pickle.dump(linear_model, open('linear_model.pkl', 'wb'))


In [ ]:
y_pred = in_season_model.predict(X_test)

In [ ]:
import math
MSE = np.square(np.subtract(y_pred,y_test)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting line:")
print(round(RMSE,1))

In [ ]:
X_test['pYards'] = in_season_model.predict(X_test)







In [ ]:
X_test['yards_gained']=y_test


X_test[['xYards/game','Yards/game','RB','WR','FB','pYards','yards_gained']].corr()

In [ ]:
X_test['error'] = X_test['pYards']-X_test['yards_gained']

In [ ]:
X_test[['xYards/game','Yards/game','pYards','error']].corr()